# Training the model
**Table of Contents**
1. [Setup](#1-setup)
2. [Dataset Preparation](#2-dataset-preparation)
3. [Training Loop](#3-training-loop)


## 1. Setup

In [2]:
# Automatic reloading
%load_ext autoreload
%autoreload 2

In [ ]:
####################
# Required Modules #
####################

# Generic/Built-in
import random

# Libs
import torch
import numpy as np
from torch.utils.data import DataLoader

# Custom
from src.data_preparation import *
from src.models import *
from src.train_eval import *

In [3]:
# Seeding
SEED = 42

# To be safe, seed all modules for full reproducibility
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)  # If using CUDA
np.random.seed(SEED)
random.seed(SEED)

## 2. Dataset Preparation

In [4]:
train_dataset, val_dataset, test_dataset, norm_stats = prepare_datasets(
    sequence_size=1000,
    stride=25,
    train_ratio=0.8,
    val_ratio=0.1,
    test_ratio=0.1,
    random_state=SEED,
    load_if_exists=True
)

🔄 Preparing dataset: Sequence Size: 1000, Stride: 25, Gap Threshold: 0.05
📂 Dataset already downloaded: data\har70.zip
📂 Dataset already extracted in data\har70plus
✅ Successfully loaded HAR70+ dataset (2259597 timestep samples).
✅ Components (HARDataset object, normalization statistics, and split indices) saved to saved_components
✅ Created train, validation, and test datasets (normalized to train set)


## 3. Training Loop

In [5]:
# Hyperparameters
batch_size = 64
learning_rate = 0.001
num_epochs = 5

In [ ]:
# Dataloaders
train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
validation_dataloader = DataLoader(val_dataset, batch_size = batch_size, shuffle = True)
test_dataloader = DataLoader(test_dataset, batch_size = batch_size, shuffle = True)

In [7]:
# Load model
model = HarGRU()
# model.load_state_dict(torch.load("models/HarGRU_2025-03-11_19-11-04/HarGRU_epoch100.pth"))
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

HarGRU model loaded on cuda.


In [8]:
training_loss_history, validation_loss_history, accuracy_history, f1_history, precision_history, recall_history = train_HAR70_model(
    model, 
    optimizer, 
    train_dataloader, 
    validation_dataloader, 
    num_epochs = num_epochs
)

Epoch [1/5] | Time: 38.11s
(Training) Loss: 0.0062
(Validation) Loss: 0.0036, Accuracy: 0.5536, F1: 0.5442, Precision: 0.5356, Recall: 0.5536
Epoch [2/5] | Time: 34.02s
(Training) Loss: 0.0031
(Validation) Loss: 0.0028, Accuracy: 0.5586, F1: 0.5486, Precision: 0.5395, Recall: 0.5586
Epoch [3/5] | Time: 32.01s
(Training) Loss: 0.0027
(Validation) Loss: 0.0025, Accuracy: 0.5599, F1: 0.5499, Precision: 0.5408, Recall: 0.5599
Epoch [4/5] | Time: 36.69s
(Training) Loss: 0.0024
(Validation) Loss: 0.0024, Accuracy: 0.5670, F1: 0.5680, Precision: 0.6202, Recall: 0.5670
Epoch [5/5] | Time: 36.69s
(Training) Loss: 0.0021
(Validation) Loss: 0.0020, Accuracy: 0.5898, F1: 0.6035, Precision: 0.7496, Recall: 0.5898
✅ Model saved: models\HarGRU_2025-03-11_23-56-06\HarGRU_epoch5.pth


In [11]:
save_training_plots_and_metric_history(training_loss_history, validation_loss_history, accuracy_history, f1_history, precision_history, recall_history, type(model).__name__)

✅ Plots saved to: results\HarGRU_2025-03-12_00-00-38
✅ Metric histories saved to: results\HarGRU_2025-03-12_00-00-38\metric_histories.pth


In [12]:
# Test metrics
loss, accuracy, f1, precision, recall, conf_matrix = evaluate_HAR70_model(model, test_dataloader)
print(f"(Test) Loss: {loss:.4f}, Accuracy: {accuracy:.4f}, F1: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")

(Test) Loss: 0.0021, Accuracy: 0.6007, F1: 0.6197, Precision: 0.7078, Recall: 0.6007
